In [2]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [3]:
#export
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
#! pip install dowhy
from dowhy import CausalModel
import dowhy.datasets, dowhy.plotter


In [5]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [6]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [7]:
%matplotlib inline

# Data Upload

In [67]:
pd_processed_logits = pd.read_csv( '../datax/ds_processed_logits/codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000_global.csv'  , index_col=0)

In [79]:
pd_processed_logits = pd.read_csv( '../datax/ds_processed_logits/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000_global.csv'  , index_col=0)

In [80]:
pd_processed_logits['model'] = '125M' #2.7B'

In [81]:
pd_processed_logits.head(5)

,ast_element,node_type,concept_median_prob,concept_min_prob,concept_max_prob,model
0,parenthesized_expression,parent,"[0.48462552019239713, 0.6351393033400654, 0.41...","[0.21738587190629916, 0.5275085141880481, 0.15...","[0.6909334224959215, 0.7828781416057609, 0.722...",125M
1,function_definition,parent,"[0.6828173127693897, 0.7061894960859931, 0.639...","[0.5879916501020537, 0.6498979980110463, 0.520...","[0.761118233763994, 0.7553395428689167, 0.6954...",125M
2,},leaf,"[0.8662500381469727, 0.9913908243179321, 0.036...","[0.8662500381469727, 0.9913908243179321, 0.036...","[0.8662500381469727, 0.9913908243179321, 0.036...",125M
3,keyword_argument,parent,"[0.4026492266481606, 0.25246529957621533, 0.47...","[0.07833874563948484, 0.0003939425666127742, 0...","[0.7971073438723882, 0.4717818001906077, 0.960...",125M
4,+=,leaf,"[0.7365161180496216, 0.9397383332252502, 0.952...","[0.7365161180496216, 0.9397383332252502, 0.952...","[0.7365161180496216, 0.9397383332252502, 0.952...",125M


In [82]:
# Concatenating all dfs
pd_processed_logits_global = pd.concat( [pd_processed_logits,pd_processed_logits_global], ignore_index=True ) #vertical concatenation

In [71]:
pd_processed_logits_global = pd_processed_logits.copy()

In [97]:
pd_processed_logits_global.head(1)

,ast_element,node_type,concept_median_prob,concept_min_prob,concept_max_prob,model,median_bootstrap_ci,median_bootstrap
0,parenthesized_expression,parent,"[0.48462552019239713, 0.6351393033400654, 0.41...","[0.21738587190629916, 0.5275085141880481, 0.15...","[0.6909334224959215, 0.7828781416057609, 0.722...",125M,"(0.5275241349831263, 0.5286622598503554, 0.000...",0.527273


In [84]:
pd_processed_logits_global.shape

In [85]:
pd_processed_logits_global[pd_processed_logits_global['model']=='125M'].shape

In [86]:
eval(pd_processed_logits_global.concept_min_prob.values[0])

[0.21738587190629916, 0.5275085141880481, 0.15305762331829303, 0.0226005338842
82674, 0.10812436127000385, 0.11863237140434128, 0.13070114548529752, 0.362353
2753920666, 0.7744759566191205, 0.47056628318624344, 0.03757901968700545, 0.12
629428931644984, 0.018835090659558774, 0.20724922525031225, 0.6354860322715704
, 0.4203966952461217, 0.09597590832378376, 0.3875050356418965, 0.1275966486965
2346, 0.21954150423407554, 0.02410807739943266, 0.07839876200471606, 0.6595408
488160077, 0.4248981452905215, 0.2941663257777691, 0.13948113763970987, 0.1565
039818603757, 0.45686424124401775, 0.6825723461687374, 0.40759929618798196, 0.
2093174636497029, 0.49563777389434666, 0.028732240200042725, 0.033711969231565
796, 0.5242675691843033, 0.04102259597857483, 0.32922801707172766, 0.388514515
00261835, 0.005771842520819821, 0.09747842256911099, 0.4939049035310745, 0.593
882717192173, 0.6465599332004786, 0.06769885094836355, 0.02011958435177803, 0.
7200596332550049, 0.6502933465289297, 0.350395795169

In [13]:
#export
def bootstrapping( np_data, np_func, size ):
    """Create a bootstrap sample given data and a function
    For instance, a bootstrap sample of means, or mediands. 
    The bootstrap replicates are a long as the original size
    we can choose any observation more than once (resampling with replacement:np.random.choice)
    """
    
    #Cleaning NaNs
    #np_data_clean = np_data[ np.logical_not( np.isnan(np_data) ) ] 
    
    #The size of the bootstrap replicate is as big as size
    #Creating the boostrap replicates as long as the orignal data size
    #This strategy might work as imputation 
    bootstrap_repl = [ np_func( np.random.choice( np_data, size=len(np_data) ) ) for i in range( size ) ]
    
    #logging.info("Covariate: " + cov) #Empirical Mean
    #logging.info("Empirical Mean: " + str(np.mean(np_data_clean))) #Empirical Mean
    #logging.info("Bootstrapped Mean: " + str( np.mean(bootstrap_repl) ) ) #Bootstrapped Mean
    
    return np.array( bootstrap_repl )

In [87]:
# Testing
test_boots = bootstrapping( 
                        np_data = eval(pd_processed_logits_global.concept_min_prob.values[0]),
                        np_func = np.min,
                        size = 500
                )

In [88]:
np.median(test_boots)

In [89]:
test_boots.shape

In [90]:
#export
#see https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
def confidence_intervals_v2(data, confidence=0.95):
    dist = NormalDist.from_samples(data)
    z = NormalDist().inv_cdf((1 + confidence) / 2.)
    h = dist.stdev * z / ((len(data) - 1) ** .5)
    return dist.mean - h, dist.mean + h, h

In [93]:
def standard_error(bootstrapped_data):
    return np.std( bootstrapped_data )

In [91]:
test_confidence = confidence_intervals_v2(data = test_boots, confidence=0.95)

In [92]:
test_confidence

In [94]:
pd_processed_logits_global['median_bootstrap_ci'] = pd_processed_logits_global.apply(
        lambda row :
                confidence_intervals_v2(
                        bootstrapping( 
                                np_data = eval(row.concept_median_prob),
                                np_func = np.median, #np.min
                                size = 500
                        )
                ) ,axis=1
    )

In [96]:
pd_processed_logits_global['median_bootstrap'] = pd_processed_logits_global.apply(
        lambda row :
                np.median(
                        bootstrapping( 
                                np_data = eval(row.concept_median_prob),
                                np_func = np.median,
                                size = 500
                        )
                ) ,axis=1
    )

In [98]:
pd_processed_logits_global['median_standard_error'] = pd_processed_logits_global.apply(
        lambda row :
                standard_error(
                        bootstrapping( 
                                np_data = eval(row.concept_median_prob),
                                np_func = np.median,
                                size = 500
                        )
                ) ,axis=1
    )

In [100]:
pd_processed_logits_global.to_csv('../datax/ds_processed_logits/codesearch_tesbed_EleutherAI-gpt-neo-all-global.csv')  

In [101]:
pd_processed_logits_global.head()

,ast_element,node_type,concept_median_prob,concept_min_prob,concept_max_prob,model,median_bootstrap_ci,median_bootstrap,median_standard_error
0,parenthesized_expression,parent,"[0.48462552019239713, 0.6351393033400654, 0.41...","[0.21738587190629916, 0.5275085141880481, 0.15...","[0.6909334224959215, 0.7828781416057609, 0.722...",125M,"(0.5275241349831263, 0.5286622598503554, 0.000...",0.527273,0.006342
1,function_definition,parent,"[0.6828173127693897, 0.7061894960859931, 0.639...","[0.5879916501020537, 0.6498979980110463, 0.520...","[0.761118233763994, 0.7553395428689167, 0.6954...",125M,"(0.654791708904859, 0.6549841036857716, 9.6197...",0.654970,0.001063
2,},leaf,"[0.8662500381469727, 0.9913908243179321, 0.036...","[0.8662500381469727, 0.9913908243179321, 0.036...","[0.8662500381469727, 0.9913908243179321, 0.036...",125M,"(0.3420513310523455, 0.34436744391007973, 0.00...",0.343633,0.013964
3,keyword_argument,parent,"[0.4026492266481606, 0.25246529957621533, 0.47...","[0.07833874563948484, 0.0003939425666127742, 0...","[0.7971073438723882, 0.4717818001906077, 0.960...",125M,"(0.5705447950339063, 0.5711377349303598, 0.000...",0.570805,0.003476
4,+=,leaf,"[0.7365161180496216, 0.9397383332252502, 0.952...","[0.7365161180496216, 0.9397383332252502, 0.952...","[0.7365161180496216, 0.9397383332252502, 0.952...",125M,"(0.5109450398547123, 0.5158243537085582, 0.002...",0.510072,0.027724


# Agreggated 

In [102]:
pd_processed_logits_global.groupby(['node_type','model']).describe()

median_bootstrap                                          \
                           count      mean       std       min       25%   
node_type model                                                            
leaf      1.3B              80.0  0.400312  0.288033  0.001604  0.156312   
          125M              83.0  0.289001  0.274917  0.000253  0.063220   
          2.7B              79.0  0.420034  0.288899  0.001083  0.208531   
parent    1.3B              85.0  0.610794  0.195136  0.007124  0.496636   
          125M              86.0  0.500235  0.212756  0.003392  0.372161   
          2.7B              85.0  0.629303  0.197265  0.010173  0.531108   

                                              median_standard_error            \
                      50%       75%       max                 count      mean   
node_type model                                                                 
leaf      1.3B   0.400177  0.607088  0.965078                  80.0  0.028756   
          125M   0.196412  0.492023  0.982604                  83.0  0.013500   
          2.7B   0.339312  0.621214  0.979225                  79.0  0.026316   
parent    1.3B   0.643457  0.731304  0.993860                  85.0  0.016802   
          125M   0.523532  0.626571  0.969144                  86.0  0.017440   
          2.7B   0.665536  0.752071  0.996733                  85.0  0.015279   

                                                                             
                      std       min       25%       50%       75%       max  
node_type model                                                              
leaf      1.3B   0.050306  0.000000  0.004180  0.010365  0.021258  0.320698  
          125M   0.022985  0.000000  0.002332  0.005808  0.011873  0.131313  
          2.7B   0.040005  0.000000  0.004821  0.012559  0.023983  0.219277  
parent    1.3B   0.027142  0.000791  0.003094  0.010562  0.019463  0.150970  
          125M   0.027703  0.000000  0.003095  0.007737  0.020356  0.158155  
          2.7B   0.020770  0.000838  0.002891  0.009542  0.017810  0.119905

## Leaf Agreggation

In [105]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='leaf') & (pd_processed_logits_global['model']=='125M') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
113,^,0.982604,0.011872
74,%,0.933755,0.003862
128,as,0.924103,0.022626
88,.,0.863973,0.001717
111,in,0.826170,0.004363
164,=,0.801372,0.001738
131,:,0.788136,0.002989
103,except,0.740165,0.005830
17,import,0.723340,0.029346
92,|=,0.720718,0.131313


In [106]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='leaf') & (pd_processed_logits_global['model']=='1.3B') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
484,%,0.965078,0.001645
423,as,0.962265,0.020620
488,.,0.931249,0.000902
376,^,0.921170,0.320698
425,in,0.889724,0.003201
490,->,0.883282,0.015454
493,=,0.873332,0.001223
371,:,0.868935,0.001948
430,],0.822611,0.003165
356,none,0.796764,0.005259


In [107]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='leaf') & (pd_processed_logits_global['model']=='2.7B') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
270,->,0.979225,0.002012
201,as,0.959854,0.022519
176,%,0.958431,0.002207
224,.,0.948615,0.000776
286,in,0.916303,0.002857
314,=,0.888076,0.001310
187,:,0.881206,0.001666
266,],0.864190,0.002384
313,none,0.822917,0.005645
245,except,0.807372,0.008121


## Parents Aggregations

In [108]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='parent') & (pd_processed_logits_global['model']=='125M') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
126,import_prefix,0.969144,0.058085
79,relative_import,0.892930,0.048136
133,aliased_import,0.860397,0.074128
39,finally_clause,0.846666,0.009892
94,elif_clause,0.834726,0.003695
55,else_clause,0.827451,0.002142
18,except_clause,0.775656,0.002943
129,try_statement,0.775513,0.002269
46,while_statement,0.773447,0.004801
5,for_statement,0.756424,0.001350


In [109]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='parent') & (pd_processed_logits_global['model']=='1.3B') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
461,import_prefix,0.993860,0.038950
336,relative_import,0.990678,0.023389
357,finally_clause,0.904283,0.006649
439,aliased_import,0.901709,0.035381
361,elif_clause,0.866562,0.003004
471,else_clause,0.859797,0.001787
470,format_specifier,0.856345,0.141797
460,for_in_clause,0.823849,0.005383
443,while_statement,0.818760,0.003656
472,dictionary_splat,0.816823,0.014303


In [110]:
pd_processed_logits_global[(pd_processed_logits_global['node_type']=='parent') & (pd_processed_logits_global['model']=='2.7B') ][['ast_element','median_bootstrap','median_standard_error']].nlargest(30, 'median_bootstrap')

,ast_element,median_bootstrap,median_standard_error
292,import_prefix,0.996733,0.027027
195,relative_import,0.991284,0.019051
291,format_specifier,0.941034,0.101362
221,finally_clause,0.904394,0.010777
329,dictionary_splat,0.901852,0.015389
284,else_clause,0.873139,0.001831
210,elif_clause,0.872139,0.002689
197,for_in_clause,0.845019,0.004919
285,aliased_import,0.833149,0.031519
321,while_statement,0.829215,0.003478


## Graphical Aggregations